In [ ]:
# import pandas
import pandas as pd

In [ ]:
# laod 'fips codes and US county' dataset
fips = pd.read_excel('drought dataset/fips.xlsx')

# view first 5 rows of the dataset
fips.head()

In [ ]:
# groupby US state to count the number of counties in each US state
fips.groupby('State').agg('count')

In [ ]:
# load validation dataset
df_validation = pd.read_csv('drought dataset/validation_timeseries/validation_timeseries.csv')

# view first 5 rows of validation dataset
df_validation.head()

In [ ]:
# load test dataset
df_test = pd.read_csv('drought dataset/test_timeseries/test_timeseries.csv')

# view first 5 rows of dataset
df_test.head()

In [ ]:
# load train dataset
df_train = pd.read_csv('drought dataset/train_timeseries/train_timeseries.csv')

# view first 5 rows of train dataset
df_train.head()

In [ ]:
# merge train, validation and test datasets to form the full dataset
frames = [df_train,df_validation,df_test]
full_dataset = pd.concat(frames)

# view summary statistics of the newly created full dataset
full_dataset.info()

In [ ]:
# sort the dataset according to fips codes and dates
full_dataset_sorted = full_dataset.sort_values(by=['fips','date'],ascending=(True,True)).reset_index(drop=True)

# verif the sorted dataset, 7671 records for each county should be present
print(full_dataset_sorted.head(7672))
print(len(full_dataset_sorted[full_dataset_sorted['fips']==1001]))

In [ ]:
# exporting the full drought dataset to a .csv file
full_dataset_sorted.to_csv('drought dataset/full_dataset_sorted.csv')

In [ ]:
# retrieve fips code data for California or 'CA' counties for filtering the full drought dataset
fips_CA = fips[fips['State']=='CA']
print(fips_CA)

# view summary statistics
print('\n',fips_CA.info())

In [ ]:
# retrieve data pertaining to California or 'CA' counties from full dataset
full_dataset_CA = full_dataset.loc[full_dataset['fips'].isin(fips_CA['FIPS'])]

# view the first 5 rows of new dataset containing data pertaining to CA counties
print(full_dataset_CA.head())

# view summary statistics of new dataset containing data pertaining to CA counties
print(full_dataset_CA.info())

In [ ]:
full_dataset_CA.describe()

In [ ]:
# sorting newly created CA counties drought dataset
full_dataset_CA_sorted = full_dataset_CA.sort_values(by=['fips','date'],ascending=(True,True)).reset_index(drop=True)

# verify the sorting, each county should have 7671 rows of data
print(full_dataset_CA_sorted.head(7672))
print('\n',len(full_dataset_CA_sorted[full_dataset_CA_sorted['fips']==6001]))

In [ ]:
# exporting the final CA counties drought dataset to a .csv file
full_dataset_CA_sorted.to_csv('drought dataset/full_dataset_CA_sorted.csv')

In [ ]:
# importing California drought dataset
cal_drought_data = pd.read_csv('drought dataset/full_dataset_CA_sorted.csv')

# drop the unnecessary column
cal_drought_data = cal_drought_data.drop(columns=['Unnamed: 0'])

# view first 5 and last 5 rows of the dataset
print(cal_drought_data.head())
print(cal_drought_data.tail())

In [ ]:
# import datetime
import datetime

In [ ]:
# convert the 'date' column to 'datetime' type
cal_drought_data['date'] = pd.to_datetime(cal_drought_data['date'])

In [ ]:
# get the list of all unique fips codes from california drought dataset
fips_code = list(cal_drought_data.fips.unique())

# view all the fips codes associated with california
print(fips_code)

# check the total number of california counties, should be 58
print(len(fips_code))

In [ ]:
# Analyze the index values of rows with drought intensity value present
y = cal_drought_data[cal_drought_data.score.notnull()].score
print(y)

# get the list of all those indexes
l = list(y.index)
print(l)

In [ ]:
# get total number of row entries for each county, with drought intensity value present
r = round(len(cal_drought_data[cal_drought_data['fips']==6001])/7)
print(r)

In [ ]:
# aggregate the last 90 days of readings for each drought reading
# define an empty dataframe
df3 = pd.DataFrame([])

# run a loop using fips codes
for fips in fips_code:
    df1 = cal_drought_data[cal_drought_data['fips']==fips].reset_index(drop=True)
    df2 = df1.drop(columns = ['date','score'])
    for i in range(r):
        if ((7*(i+1))+3-90)<0:
            agg_data = pd.DataFrame(df2[0:((7*(i+1))+3)].agg("mean",axis="rows"))
            df3 = df3.append(agg_data.transpose())
        else:
            agg_data = pd.DataFrame(df2[((7*(i+1))+3-90):((7*(i+1)+3))].agg("mean",axis="rows"))
            df3 = df3.append(agg_data.transpose())

In [ ]:
# view the new 90 days aggregated dataframe
print(df3)

# view the total number of entries in new dataset
print(len(df3))

In [ ]:
# add the drought intensity score column to the 90 days aggregated dataframe
df_score = list(y)
df3['score'] = df_score

# drop unnecessary columns
df4 = df3.drop(columns=['Unnamed: 0'])
print(df4)

In [ ]:
# export the 90 days aggregated data
df4.to_csv('drought dataset/ninety_days_aggregated_data_ca_drought.csv')

In [ ]:
# retrieve data for finding correlations
# drop the 'fips' column
data = df4.drop(columns=['fips'])

# find the correlation matrix
data_corr = data.corr()
print(data_corr)

In [ ]:
# plot the correlation matrix heatmap
ax = sns.heatmap(data_corr,vmin=-1,vmax=1,center=0,cmap=sns.diverging_palette(20,220,n=200),square=True)